# Rosters

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
DATA_DIR = Path.cwd() / 'data'

In [3]:
def filter_on_team(t: pd.DataFrame, team_id: int) -> pd.DataFrame:
    return t[t['fantasy_team_id'] == team_id].sort_values(['pos', 'current_position'], ascending=[True, False])

In [4]:
lineups = pd.read_csv(DATA_DIR / 'lineups_week_01.csv')
depth_chart = pd.read_csv(DATA_DIR / 'depth_chart_2021.csv')

name_fixes = {
    # ESPN name to the Huddle name
    'William Fuller V': 'Will Fuller',
    'Darrell Henderson Jr.': 'Darrell Henderson',
    'Allen Robinson II': 'Allen Robinson',
    'Russell Gage': 'Russell Gage Jr.',
    'DJ Moore': 'D.J. Moore',
    'Marvin Jones Jr.': 'Marvin Jones',
    'Ronald Jones II': 'Ronald Jones',
}

adjusted_names = lineups.copy()
adjusted_names['player_name'] = adjusted_names['player_name'].apply(lambda s: name_fixes.get(s, s))

# filter out defense
# adjusted_names = adjusted_names[adjusted_names['pos'] != 'D/ST']

cols = ['player', 'depth', 'qualifiers']
df = pd.merge(adjusted_names, depth_chart[cols], left_on='player_name', right_on='player', how='left')

# make sure everything joined corrected
assert df['pos'].isnull().sum() == 0
assert ((df['player'].isnull()) & (df['pos'] != 'D/ST')).sum() == 0
# assert len(adjusted_names) == len(df)

df['depth'] = df['depth'].astype('Int64')

df.head(2)

,player_id,player_name,pos,team,projected_points,current_position,QB,RB,WR,TE,...,DST,week,fantasy_team_id,fantasy_team_name,opponent_team_id,opponent_team_name,home_away,player,depth,qualifiers
0,3043078,Derrick Henry,RB,TEN,16.55,RB,False,True,False,False,...,False,1,16,Draftin Herbert First Pick,19,Federal Way Yu,home,Derrick Henry,1,NaN
1,4259545,D'Andre Swift,RB,DET,9.95,RB,False,True,False,False,...,False,1,16,Draftin Herbert First Pick,19,Federal Way Yu,home,D'Andre Swift,1,NaN


### league teams

In [5]:
df[['fantasy_team_id', 'fantasy_team_name']].drop_duplicates().sort_values('fantasy_team_id')

,fantasy_team_id,fantasy_team_name
45,1,Fully Maccinated
60,2,Rippin' Swigs
167,3,Seattle Fantasy Team
136,10,Spenny Willy
106,11,Raiders Are Good
121,12,Herb Your Enthusiasm
75,13,Too Many Cooks
151,15,Elite starts with Eli
0,16,Draftin Herbert First Pick
90,17,Scooty Lewis and the News


### player position versus current roster position

* 2 RB x 12 = 24
* 3 WR x 12 = 36

In [6]:
pd.pivot_table(df, index='pos', columns='current_position', values='player_id', aggfunc='count', margins=True).fillna(0).astype(int)

current_position,BE,D/ST,IR,K,QB,RB,RB/WR/TE,TE,WR,All
pos,,,,,,,,,,
D/ST,2,12,0,0,0,0,0,0,0,14
K,0,0,0,12,0,0,0,0,0,12
QB,7,0,1,0,12,0,0,0,0,20
RB,21,0,0,0,0,24,5,0,0,50
TE,6,0,0,0,0,0,0,12,0,18
WR,24,0,1,0,0,0,7,0,36,68
All,60,12,2,12,12,24,12,12,36,182


### IR slots in use

In [7]:
df[df['current_position'] == 'IR']

,player_id,player_name,pos,team,projected_points,current_position,QB,RB,WR,TE,...,DST,week,fantasy_team_id,fantasy_team_name,opponent_team_id,opponent_team_name,home_away,player,depth,qualifiers
96,14876,Ryan Tannehill,QB,TEN,18.24,IR,True,False,False,False,...,False,1,17,Scooty Lewis and the News,11,Raiders Are Good,home,Ryan Tannehill,1,NaN
157,2976316,Michael Thomas,WR,NO,0.00,IR,False,False,True,False,...,False,1,15,Elite starts with Eli,3,Seattle Fantasy Team,home,Michael Thomas,1,Physically Unable to Perform


## Handcuffs

>  A fantasy handcuff is a backup that will likely take over for a team's starter in the event of an injury. A common strategy on draft day is to stash the backups of the prominent RBs you've drafted.

https://www.fantasypros.com/nfl/running-back-handcuffs.php?scoring=HALF&leaguesize=12

### teams with handcuffs picked up

or the 2 and 3 in the case of PIT

In [8]:
depths_agg = df.groupby(['fantasy_team_name', 'pos', 'team'])[['player_name', 'depth']].agg(lambda d: set(d)).reset_index()
depths_agg[depths_agg['depth'].apply(len) > 1].sort_values('fantasy_team_name')

,fantasy_team_name,pos,team,player_name,depth
36,Federal Way Yu,RB,HOU,"{Phillip Lindsay, David Johnson}","{1, 2}"
37,Federal Way Yu,RB,MIN,"{Alexander Mattison, Dalvin Cook}","{1, 2}"
71,Herb Your Enthusiasm,WR,PIT,"{Diontae Johnson, Chase Claypool}","{2, 3}"
79,Raiders Are Good,RB,OAK,"{Josh Jacobs, Kenyan Drake}","{1, 2}"
112,Scooty Lewis and the News,WR,ATL,"{Russell Gage Jr., Calvin Ridley}","{1, 2}"
151,Team Daktus Jack,RB,NYG,"{Saquon Barkley, Devontae Booker}","{1, 2}"
157,Team Daktus Jack,WR,LAC,"{Keenan Allen, Mike Williams}","{1, 2}"


### Teams without the RB handcuff

In [9]:
m = (depths_agg['depth'].apply(len) == 1) & (depths_agg['pos'] == 'RB') & (depths_agg['depth'].apply(lambda s: list(s)[0] == 1))
depths_agg[m].sort_values(['fantasy_team_name', 'pos', 'team'])

,fantasy_team_name,pos,team,player_name,depth
3,Draftin Herbert First Pick,RB,BUF,{Zack Moss},{1}
4,Draftin Herbert First Pick,RB,CHI,{David Montgomery},{1}
6,Draftin Herbert First Pick,RB,DET,{D'Andre Swift},{1}
7,Draftin Herbert First Pick,RB,TEN,{Derrick Henry},{1}
18,Elite starts with Eli,RB,IND,{Jonathan Taylor},{1}
19,Elite starts with Eli,RB,KC,{Clyde Edwards-Helaire},{1}
48,Fully Maccinated,RB,BAL,{Gus Edwards},{1}
49,Fully Maccinated,RB,CAR,{Christian McCaffrey},{1}
50,Fully Maccinated,RB,LAR,{Darrell Henderson},{1}
51,Fully Maccinated,RB,NE,{Damien Harris},{1}


### teams/players of interest

This is based on the current roster so any depth players (i.e. RB2) that don't show up are currently Free Agents

Tier 1 and 2 according to FantasyPros - https://www.fantasypros.com/nfl/rankings/half-point-ppr-rb-cheatsheets.php

In [10]:
teams = ['CAR', 'MIN', 'NO', 'TEN', 'DAL', 'GB', 'NYG', 'IND', 'CLE', 'LAC']
subset = ['fantasy_team_name', 'player_name', 'pos', 'team', 'projected_points', 'current_position', 'depth']
for team in teams:
    m = (df['team'].str.contains(team)) & (df['pos'] == 'RB')
    display(df[m][subset].sort_values('depth'))

,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
45,Fully Maccinated,Christian McCaffrey,RB,CAR,18.4,RB,1


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
15,Federal Way Yu,Dalvin Cook,RB,MIN,18.64,RB,1
29,Federal Way Yu,Alexander Mattison,RB,MIN,6.53,BE,2


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
137,Spenny Willy,Alvin Kamara,RB,NO,14.95,RB,1


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
0,Draftin Herbert First Pick,Derrick Henry,RB,TEN,16.55,RB,1


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
60,Rippin' Swigs,Ezekiel Elliott,RB,DAL,12.83,RB,1
103,Scooty Lewis and the News,Tony Pollard,RB,DAL,5.32,BE,2


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
61,Rippin' Swigs,Aaron Jones,RB,GB,13.48,RB,1
174,Seattle Fantasy Team,AJ Dillon,RB,GB,7.65,BE,2


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
30,Team Daktus Jack,Saquon Barkley,RB,NYG,12.22,RB,1
43,Team Daktus Jack,Devontae Booker,RB,NYG,5.02,BE,2


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
152,Elite starts with Eli,Jonathan Taylor,RB,IND,14.36,RB,1


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
122,Herb Your Enthusiasm,Nick Chubb,RB,CLE,14.14,RB,1
18,Federal Way Yu,Kareem Hunt,RB,CLE,9.60,RB,2


,fantasy_team_name,player_name,pos,team,projected_points,current_position,depth
168,Seattle Fantasy Team,Austin Ekeler,RB,LAC,12.45,RB,1


### free agent handcuffs

In [11]:
handcuff_fa_teams = ['CAR', 'NO', 'TEN', 'IND', 'LAC']
for team in handcuff_fa_teams:
    m = (depth_chart['team'].str.contains(team)) & (depth_chart['pos'] == 'RB')
    display(depth_chart[m].sort_values('depth'))

,team,pos,depth,player,qualifiers
68,CAR,RB,1,Christian McCaffrey,NaN
69,CAR,RB,2,Chuba Hubbard,Rookie
70,CAR,RB,3,Royce Freeman,NaN


,team,pos,depth,player,qualifiers
349,NO,RB,1,Alvin Kamara,NaN
350,NO,RB,2,Tony Jones Jr.,NaN
351,NO,RB,3,Dwayne Washington,NaN
352,NO,RB,4,Ty Montgomery,NaN


,team,pos,depth,player,qualifiers
478,TEN,RB,1,Derrick Henry,NaN
479,TEN,RB,2,Mekhi Sargent,NaN
480,TEN,RB,3,Jeremy McNichols,NaN
481,TEN,RB,4,Darrynton Evans,Injured Reserve w/ opportunity to return


,team,pos,depth,player,qualifiers
202,IND,RB,1,Jonathan Taylor,NaN
203,IND,RB,2,Nyheim Hines,NaN
204,IND,RB,3,Jordan Wilkins,NaN
205,IND,RB,4,Marlon Mack,NaN


,team,pos,depth,player,qualifiers
265,LAC,RB,1,Austin Ekeler,NaN
266,LAC,RB,2,Justin Jackson,NaN
267,LAC,RB,3,Larry Rountree,Rookie
268,LAC,RB,4,Joshua Kelley,NaN
